In [1]:
import csv
import re
import pandas as pd

In [2]:
def parse_pair(text):
    text = re.sub(r"\@", "", text)
    pairs = text.split("\t")
    pairs_list = list()
    for pair in pairs:
        pairs_list.append(tuple(pair.split(",")))
    return pairs_list

In [3]:
pairs_list = list()
label_list = list()
with open('train_return_pair.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
        label_list.append(row[1])
        pairs_list.append(parse_pair(row[2]))

In [4]:
pair_set = set()
for pairs in pairs_list:
    for pair in pairs:
        if len(pair) == 3:
            pair_set.add((pair[0], pair[1]))

In [5]:
pair_list = list(pair_set)
pair_dict = {}
for i in range(len(pair_list)):
    pair_dict[pair_list[i]] = i

In [6]:
train_df = pd.DataFrame(columns=pair_list)
columns_num = len(train_df.columns)
for i in range(len(pairs_list)):
    pairs = pairs_list[i]
    row = [-1] * columns_num
    for pair in pairs:
        if len(pair) != 3:
            continue
        value = pair[2]
        pair_label = (pair[0], pair[1])
        if pair_label in pair_dict:
            row[pair_dict[pair_label]] = value
    train_df.loc[i] = row

KeyboardInterrupt: 

In [7]:
print(len(train_df))

9876


In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_df.as_matrix(), label_list[0:9876])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [14]:
pairs_test_list = list()
label_test_list = list()
with open('test_return_pair.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
        label_test_list.append(row[1])
        pairs_test_list.append(parse_pair(row[2]))

In [17]:
test_df = pd.DataFrame(columns=pair_list)
columns_num = len(test_df.columns)
for i in range(len(pairs_test_list)):
    pairs = pairs_test_list[i]
    row = [-1] * columns_num
    for pair in pairs:
        if len(pair) != 3:
            continue
        value = pair[2]
        pair_label = (pair[0], pair[1])
        if pair_label in pair_dict:
            row[pair_dict[pair_label]] = value
    test_df.loc[i] = row

In [19]:
y_pred = clf.predict(test_df.as_matrix())

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(label_test_list, y_pred)

0.52392133842089816

In [22]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(train_df.as_matrix(), label_list[0:9876]).predict(test_df.as_matrix())
accuracy_score(label_test_list, y_pred)

0.50102729674200175

In [24]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(train_df.as_matrix(), label_list[0:9876]).predict(test_df.as_matrix())
accuracy_score(label_test_list, y_pred)

0.54241267977692986